In [1]:
import tensorflow as tf
import glob

In [2]:
import pandas as pd 
import os
import numpy as np 
from tensorflow.keras.preprocessing import image 
from tensorflow.keras import regularizers, utils

import numpy as np 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, AveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from skimage.segmentation import mark_boundaries 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
multiple_labels_path = "labels/multiple_food.txt"
labels = pd.DataFrame(columns=['Filenames', 'labels'])
categories = [str(x) for x in range(1,101)]

In [4]:
with open(multiple_labels_path) as f:
    contents = f.read().split('\n')
    for content in contents[1:]:
        data = content.split(" ")
        filename = f'{data[0]}.jpg'
        label = data[1:]
        cleaned_label = [x for x in label if x != '']
        labels = labels.append({'Filenames': filename, 'labels': cleaned_label}, ignore_index=True)

In [5]:
labels.head(5)

,Filenames,labels
0,1.jpg,"[1, 42]"
1,9.jpg,"[1, 24]"
2,14.jpg,"[1, 36]"
3,19.jpg,"[1, 69, 70]"
4,22.jpg,"[1, 36, 67, 70]"


In [6]:
train_df = pd.DataFrame(columns=['Filenames', 'labels'])
# Traverse entire directory and update labels
for root, dirs, files in os.walk("./UECFOOD100_CROP"):
    if root.startswith('./UECFOOD100_CROP/train/'):
        label = root.split('/')[-1]
        for file in files:
            if file.endswith(".jpg"):
                if len(labels.loc[labels['Filenames'] == file]) > 0:
                    if len(train_df.loc[train_df['Filenames'] == file]) > 0:
                        continue
                    multi_labels = list(labels.loc[labels['Filenames'] == file].labels)[0]
                    train_df = train_df.append({'Filenames': f'{label}/{file}', 'labels': multi_labels}, ignore_index=True) 
                else:
                    train_df = train_df.append({'Filenames': f'{label}/{file}', 'labels': [label]}, ignore_index=True)

In [7]:
train_df.head(5)

,Filenames,labels
0,61/6170.jpg,[61]
1,61/6158.jpg,[61]
2,61/6159.jpg,[61]
3,61/6171.jpg,[61]
4,61/6165.jpg,[61]


In [8]:
test_df = pd.DataFrame(columns=['Filenames', 'labels'])
# Traverse entire directory and update labels
for root, dirs, files in os.walk("./UECFOOD100_CROP"):
    if root.startswith('./UECFOOD100_CROP/test/'):
        label = root.split('/')[-1]
        for file in files:
            if file.endswith(".jpg"):
                if len(labels.loc[labels['Filenames'] == file]) > 0:
                    if len(test_df.loc[test_df['Filenames'] == file]) > 0:
                        continue
                    multi_labels = list(labels.loc[labels['Filenames'] == file].labels)[0]
                    test_df = test_df.append({'Filenames': f'{label}/{file}', 'labels': multi_labels}, ignore_index=True) 
                else:
                    test_df = test_df.append({'Filenames': f'{label}/{file}', 'labels': [label]}, ignore_index=True)

In [9]:
print(train_df.shape)
print(test_df.shape)

(11514, 2)
(2902, 2)


Reference: https://vijayabhaskar96.medium.com/multi-label-image-classification-tutorial-with-keras-imagedatagenerator-cd541f8eaf24

In [11]:
# Multilabel - train
datagen=ImageDataGenerator(rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.2)
test_gen=ImageDataGenerator(rescale=1./255)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_generator=datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="./UECFOOD100_CROP/train/",
    x_col="Filenames",
    y_col="labels",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    classes=categories,
    target_size=(128,128),
    subset="training")

validation_generator=datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="./UECFOOD100_CROP/train",
    x_col="Filenames",
    y_col="labels",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    classes=categories,
    target_size=(128,128),
    subset="validation")

test_generator=test_gen.flow_from_dataframe(
    dataframe=test_df,
    directory="./UECFOOD100_CROP/test",
    x_col="Filenames",
    y_col="labels",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(128,128))

Found 9212 validated image filenames belonging to 100 classes.
Found 2302 validated image filenames belonging to 100 classes.
Found 2902 validated image filenames belonging to 100 classes.


In [23]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.applications import ResNet50
import tensorflow.keras
import tensorflow.keras.metrics

## Inception

In [25]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

base_model = InceptionV3(weights='imagenet', include_top=False)

for layer in inception.layers:
    layer.trainable = False
    
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(2048,activation='relu')(x)
x = Dense(2048,activation='relu')(x)
x = Dense(2048,activation='relu')(x)
output = Dense(100,kernel_regularizer=regularizers.l2(0.005), activation='sigmoid')(x)
# output = Dense(100,kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)
model = Model(inputs=inception.input, outputs=output)
model.compile(loss = 'binary_crossentropy', optimizer = Adam(learning_rate=0.001), metrics = ['categorical_accuracy', 'binary_accuracy', 
                                                                             'top_k_categorical_accuracy'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_282 (Conv2D)             (None, None, None, 3 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_285 (BatchN (None, None, None, 3 96          conv2d_282[0][0]                 
__________________________________________________________________________________________________
activation_282 (Activation)     (None, None, None, 3 0           batch_normalization_285[0][0]    
____________________________________________________________________________________________

Baseline train and test accuracy

In [26]:
model.evaluate(train_generator)

288/288 [==============================] - 154s 534ms/step - loss: 1.7739 - categorical_accuracy: 0.0047 - binary_accuracy: 0.4771 - top_k_categorical_accuracy: 0.0329


[1.7738800048828125,
 0.004667824599891901,
 0.47711682319641113,
 0.032891880720853806]

In [27]:
model.evaluate(test_generator)

91/91 [==============================] - 43s 475ms/step - loss: 1.7746 - categorical_accuracy: 0.0079 - binary_accuracy: 0.4746 - top_k_categorical_accuracy: 0.0438


[1.774566888809204,
 0.007925568148493767,
 0.47456932067871094,
 0.043762922286987305]

In [28]:
#early stopping to monitor the validation loss and avoid overfitting
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)

#reducing learning rate on plateau
rlrop = ReduceLROnPlateau(monitor='val_loss', mode='min', patience= 5, factor= 0.5, min_lr= 1e-6, verbose=1)

#With training and validation data
history1 = model.fit(train_generator,
                    validation_data=validation_generator,
                    epochs=25,
                    verbose=True,  
                    callbacks=[early_stop, rlrop]
)

Epoch 1/25
288/288 [==============================] - 247s 858ms/step - loss: 0.1595 - categorical_accuracy: 0.1592 - binary_accuracy: 0.9849 - top_k_categorical_accuracy: 0.3545 - val_loss: 0.0897 - val_categorical_accuracy: 0.0456 - val_binary_accuracy: 0.9875 - val_top_k_categorical_accuracy: 0.0743 - lr: 0.0010
Epoch 2/25
288/288 [==============================] - 235s 816ms/step - loss: 0.0585 - categorical_accuracy: 0.2485 - binary_accuracy: 0.9872 - top_k_categorical_accuracy: 0.5144 - val_loss: 0.0912 - val_categorical_accuracy: 0.0469 - val_binary_accuracy: 0.9865 - val_top_k_categorical_accuracy: 0.0912 - lr: 0.0010
Epoch 3/25
288/288 [==============================] - 254s 881ms/step - loss: 0.0544 - categorical_accuracy: 0.2890 - binary_accuracy: 0.9874 - top_k_categorical_accuracy: 0.5752 - val_loss: 0.0940 - val_categorical_accuracy: 0.0569 - val_binary_accuracy: 0.9865 - val_top_k_categorical_accuracy: 0.0830 - lr: 0.0010
Epoch 4/25
288/288 [=============================

## VGG

In [31]:
from tensorflow.keras.applications.vgg16 import VGG16
optimizer = Adam(lr=0.001)

base_model = VGG16(weights='imagenet', include_top=False)

for layer in vgg.layers:
    layer.trainable = False
    
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(2048,activation='relu')(x)
x = Dense(2048,activation='relu')(x)
x = Dense(2048,activation='relu')(x)
x = Dense(2048,activation='relu')(x)

output = Dense(100,kernel_regularizer=regularizers.l2(0.005), activation='sigmoid')(x)
# output = Dense(100,kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(loss = 'binary_crossentropy', optimizer = Adam(learning_rate=0.001), metrics = ['categorical_accuracy', 'binary_accuracy', 
                                                                             'top_k_categorical_accuracy'])
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0   

In [32]:
model.evaluate(train_generator)
model.evaluate(test_generator)

91/91 [==============================] - 351s 4s/step - loss: 1.6562 - categorical_accuracy: 0.0107 - binary_accuracy: 0.4815 - top_k_categorical_accuracy: 0.0593


[1.6562414169311523,
 0.010682287625968456,
 0.48147135972976685,
 0.05926946923136711]

In [33]:
#With training and validation data
history2 = model.fit(train_generator,
                    validation_data=validation_generator,
                    epochs=25,
                    verbose=True,  
                    callbacks=[early_stop, rlrop]
)

Epoch 1/25
288/288 [==============================] - 1448s 5s/step - loss: 0.1571 - categorical_accuracy: 0.0992 - binary_accuracy: 0.9848 - top_k_categorical_accuracy: 0.2378 - val_loss: 0.1057 - val_categorical_accuracy: 0.0556 - val_binary_accuracy: 0.9876 - val_top_k_categorical_accuracy: 0.1043 - lr: 0.0010
Epoch 2/25
288/288 [==============================] - 1106s 4s/step - loss: 0.0645 - categorical_accuracy: 0.1446 - binary_accuracy: 0.9871 - top_k_categorical_accuracy: 0.3193 - val_loss: 0.0875 - val_categorical_accuracy: 0.0456 - val_binary_accuracy: 0.9872 - val_top_k_categorical_accuracy: 0.0821 - lr: 0.0010
Epoch 3/25
288/288 [==============================] - 1089s 4s/step - loss: 0.0594 - categorical_accuracy: 0.1761 - binary_accuracy: 0.9872 - top_k_categorical_accuracy: 0.3998 - val_loss: 0.0871 - val_categorical_accuracy: 0.0582 - val_binary_accuracy: 0.9876 - val_top_k_categorical_accuracy: 0.0895 - lr: 0.0010
Epoch 4/25
288/288 [==============================] - 1